In [5]:
#import sys
#!{sys.executable} -m pip install torch gym_conservation "ray[rllib, tune]"

In [7]:
import gym
import gym_climate
import gym_conservation
from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

In [8]:
## Training
#env = make_vec_env("ays-v0", n_envs=4)
#model = A2C("MlpPolicy", env, verbose=1,tensorboard_log="/var/log/tensorboard/carl")
#model.learn(total_timesteps=10)
#model.save("ays-v0-A2C")

# Evaluation
model A2C.load("ays-v0-A2c.zip")
eval_env = Monitor(gym.make("ays-v0"))
score = evaluate_policy(model, eval_env, n_eval_episodes=10)
score


In [35]:

import argparse
import gym
import numpy as np
import os

import ray
from ray import tune
from ray.rllib import agents
import torch

import gym_conservation
import gym_fishing

## rllib ignores gym registered names, need to register manually:
## note these envs were not written to take a single parameter dictionary ("config")
tune.register_env("conservation-v6", lambda config: gym_conservation.envs.NonStationaryV6())
tune.register_env("conservation-v5", lambda config: gym_conservation.envs.NonStationaryV5())
tune.register_env("fishing-v0", lambda config: gym_fishing.envs.FishingEnv())
tune.register_env("fishing-v1", lambda config: gym_fishing.envs.FishingCtsEnv())

os.environ["RLLIB_NUM_GPUS"] = str(torch.cuda.device_count())
os.environ.get("RLLIB_NUM_GPUS")


'1'

In [30]:
# Read in an ExperimentAnalysis from a json state file

#result = "~/ray_results/ARS/"
result = "~/ray_results/TD3/"

analysis = tune.ExperimentAnalysis(experiment_checkpoint_path=result)
analysis.get_best_trial(metric="episode_reward_mean", mode="max").last_result



{'episode_reward_mean': 159.73949,
 'episode_len_mean': nan,
 'timesteps_this_iter': 16032,
 'info': {'weights_norm': 638.5123,
  'weights_std': 0.09775411,
  'grad_norm': 3606.4155,
  'update_ratio': 0.023772934,
  'episodes_this_iter': 32,
  'episodes_so_far': 6016},
 'done': True,
 'timesteps_total': 3010912,
 'episodes_total': None,
 'training_iteration': 188,
 'trial_id': 'a2531_00000',
 'experiment_id': '56fb169c732f4b98b95ebe4e1c144b85',
 'date': '2022-02-16_18-11-15',
 'timestamp': 1645035075,
 'time_this_iter_s': 4.648577690124512,
 'time_total_s': 922.7278463840485,
 'pid': 1930718,
 'hostname': 'f3c1977db4f9',
 'node_ip': '172.18.0.5',
 'config': {'num_workers': 2,
  'num_envs_per_worker': 1,
  'create_env_on_driver': False,
  'rollout_fragment_length': 200,
  'batch_mode': 'truncate_episodes',
  'gamma': 0.99,
  'lr': 0.0001,
  'train_batch_size': 200,
  'model': {'_use_default_native_models': False,
   '_disable_preprocessor_api': False,
   'fcnet_hiddens': [256, 256],
   

In [39]:
# extract the config
config = analysis.get_best_trial(metric="episode_reward_mean", mode="max").last_result["config"]
config

{'num_workers': 2,
 'num_envs_per_worker': 1,
 'create_env_on_driver': False,
 'rollout_fragment_length': 200,
 'batch_mode': 'truncate_episodes',
 'gamma': 0.99,
 'lr': 0.0001,
 'train_batch_size': 200,
 'model': {'_use_default_native_models': False,
  '_disable_preprocessor_api': False,
  'fcnet_hiddens': [256, 256],
  'fcnet_activation': 'tanh',
  'conv_filters': None,
  'conv_activation': 'relu',
  'post_fcnet_hiddens': [],
  'post_fcnet_activation': 'relu',
  'free_log_std': False,
  'no_final_linear': False,
  'vf_share_layers': True,
  'use_lstm': False,
  'max_seq_len': 20,
  'lstm_cell_size': 256,
  'lstm_use_prev_action': False,
  'lstm_use_prev_reward': False,
  '_time_major': False,
  'use_attention': False,
  'attention_num_transformer_units': 1,
  'attention_dim': 64,
  'attention_num_heads': 1,
  'attention_head_dim': 32,
  'attention_memory_inference': 50,
  'attention_memory_training': 50,
  'attention_position_wise_mlp_dim': 32,
  'attention_init_gru_gate_bias': 2.0,


(Worker pid=2346234) 2022-02-17 05:58:06,788	INFO torch_policy.py:165 -- TorchPolicy (worker=local) running on CPU.


In [38]:
model = agents.ars.ARSTrainer(config)

2022-02-17 05:57:51,114	WARNING deprecation.py:46 -- DeprecationWarning: `simple_optimizer` has been deprecated. This will raise an error in the future!
2022-02-17 05:57:51,115	WARNING deprecation.py:46 -- DeprecationWarning: `simple_optimizer` has been deprecated. This will raise an error in the future!


AssertionError: 

(create_shared_noise pid=2346234) /home/cboettig/.local/share/virtualenvs/rl-ray-demo-y9hdSGSh/lib/python3.7/site-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
(create_shared_noise pid=2346234)   for external in metadata.entry_points().get(self.group, []):
(pid=2346233) /home/cboettig/.local/share/virtualenvs/rl-ray-demo-y9hdSGSh/lib/python3.7/site-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
(pid=2346233)   for external in metadata.entry_points().get(self.group, []):
(Worker pid=2346233) 2022-02-17 05:58:05,550	INFO torch_policy.py:165 -- TorchPolicy (worker=local) running on CPU.


In [ ]:
model.restore(result)

In [19]:
path = os.path.relpath("rllib_PPO_fishing-v0/")

model.load_checkpoint("rllib_PPO_fishing-v0/checkpoint_000004/checkpoint-4")

In [ ]:

# Evaluate the trained Trainer (and render each timestep to the shell's output).
trainer.evaluate()



{'evaluation': {'episode_reward_max': 1.2274416977413574,
  'episode_reward_min': 0.75,
  'episode_reward_mean': 0.8675837622445943,
  'episode_len_mean': 3.8,
  'episode_media': {},
  'episodes_this_iter': 10,
  'policy_reward_min': {},
  'policy_reward_max': {},
  'policy_reward_mean': {},
  'custom_metrics': {},
  'hist_stats': {'episode_reward': [0.80625,
    0.80625,
    1.2274416977413574,
    0.75,
    0.75,
    1.0980312748711576,
    0.80625,
    0.80625,
    0.8191146498334291,
    0.80625],
   'episode_lengths': [2, 2, 9, 1, 1, 14, 2, 2, 3, 2]},
  'sampler_perf': {'mean_raw_obs_processing_ms': 0.3598704934120178,
   'mean_inference_ms': 1.170910894870758,
   'mean_action_processing_ms': 0.11758009592692058,
   'mean_env_wait_ms': 0.10723620653152466,
   'mean_env_render_ms': 0.0},
  'off_policy_estimator': {},
  'timesteps_this_iter': 0}}

In [ ]:
trainer.predict = trainer.compute_action
eval_env = Monitor(gym.make("fishing-v1"))
score = evaluate_policy(trainer, eval_env, n_eval_episodes=10)#


2022-02-08 01:24:23,846	WARNING deprecation.py:46 -- DeprecationWarning: `compute_action` has been deprecated. Use `Trainer.compute_single_action()` instead. This will raise an error in the future!


ValueError: not enough values to unpack (expected 2, got 1)

E0208 04:37:45.148504575 3607986 backup_poller.cc:134]       Run client channel backup poller: {"created":"@1644295065.148450202","description":"pollset_work","file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":320,"referenced_errors":[{"created":"@1644295065.148445954","description":"Bad file descriptor","errno":9,"file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":950,"os_error":"Bad file descriptor","syscall":"epoll_wait"}]}
